1.1 Install relevant frameworks

In [ ]:
#!pip install yellowbrick  
#!pip install -U sentence-transformers

1.2 Import relevant Libraries, frameworks

In [73]:
# Importing relevant frameworks and libraries
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer
from sentence_transformers import SentenceTransformer, util

1.3 Load Data

In [82]:
# dataset sources
# amazon jobs data : https://www.kaggle.com/datasets/niyamatalmass/google-job-skills
# google jobs data : https://www.kaggle.com/datasets/atahmasb/amazon-job-skills
amazon = pd.read_csv("amazon_jobs_dataset.csv", index_col = 0)
google = pd.read_csv("job_skills.csv", index_col=0)

In [83]:
amazon.head(2)

,Title,location,Posting_date,DESCRIPTION,BASIC QUALIFICATIONS,PREFERRED QUALIFICATIONS
0,Software Development Manager,"US, WA, Seattle","March 1, 2018","You are an experienced hands-on manager with a background in developing and delivering software that simplifies solutions for a broad set of related problems. You have a strong technical ability, great communication skills, and know how to inspire your team to achieve outstanding results in a fast paced environment. You have a track record of building and leading strong teams of talented engineers and you enjoy working with smart people. In short - you are a skilled leader capable of getting great things done for your customers. If it sounds like you - then read on!As one of the largest companies in the cloud computing space across the world, Amazon Web Services (AWS) offers dozens of services and processes billions of usage records. Over the past 10 years, Amazon has become known across the globe as the most trusted company on the internet.The systems we build interpret and process a very high volume stream of events; often bundled with low-latency/real-time analysis to enable security and fraud detection.AWS Bill Generation team manages the computation engine responsible for generating charges to the customers. Every time any AWS service meters a usage by a customer, a record makes its way to the our computation engine. At the simplest level, the bill computation engine generates bill line items corresponding to usage. Every day the bill computation engine processes hundreds of millions of records. The computation engine is architected to enable us to scale to this computational load and grow with the rapidly growing AWS business.There are numerous highly varied scenarios that the bill computation engine handles: the common most one is that of tiering, i.e. charging customers at different rates based on usage. These tiers may be computed across multiple dimensions: across geographical regions, across accounts, or across services. One of the key goals of the bill computation engine is to build a highly scalable, general purpose multi-dimensional counter mechanism that enables us to support a very flexible set of pricing models. Another key goal is to make the bill computation engine ""declarative"". We want to make it super easy for sellers of cloud services to create complex pricing plans with minimal or no code changes whatsoever.Key Responsibilities:· Adopt overall AWS plans for AWS Commerce Platform and define your own team's future direction.· · Manage projects that deliver on commitments to our customers.· · Manage day-to-day activities of engineering team using Agile/Scrum methodology.· · Manage team's resources: hire, mentor, and grow the engineering team.· · Work closely with engineers to architect and develop the best technical solutions. Be a hands-on technical leader.· · Report on status of development, quality, operations, and system performance to management.","· Proven track record of hiring and managing high performing engineering teams· Breadth and depth of experience building and managing software systems and teams· · Excellent written and verbal communication skills with the ability to present complex technical information in a clear and concise manner to executives and non-technical leaders· · Willingness to roll up your sleeves and do what's necessary to drive your projects to completion· · 6+ years of experience in Software development and 4+ years as a Development Manager· · Strong technical credentials, with expertise in OOP, Java or C++· · Experience with Linux/Unix systems· · Experience with SQL and No SQL data stores· · Bachelor's Degree in Computer Science or related degree","· Experience building extremely high volume and highly scalable web services· · Eperience building highly available systems and operating 24x7 services· · Experience with distributed systems, consistent hashing, distributed locking, replication, and load balancing· · Master’s 

In [84]:
google.head(2)

,Title,Category,Location,Responsibilities,Minimum Qualifications,Preferred Qualifications
Company,,,,,,
Google,Google Cloud Program Manager,Program Management,Singapore,"Shape, shepherd, ship, and show technical programs designed to support the work of Cloud Customer Engineers and Solutions Architects.\nMeasure and report on key metrics tied to those programs to identify any need to change course, cancel, or scale the programs from a regional to global platform.\nCommunicate status and identify any obstacles and paths for resolution to stakeholders, including those in senior roles, in a transparent, regular, professional and timely manner.\nEstablish expectations and rationale on deliverables for stakeholders and program contributors.\nProvide program performance feedback to teams in Product, Engineering, Sales, and Marketing (among others) to enable efficient cross-team operations.","BA/BS degree or equivalent practical experience.\n3 years of experience in program and/or project management in cloud computing, enterprise software and/or marketing technologies.","Experience in the business technology market as a program manager in SaaS, cloud computing, and/or emerging technologies.\nSignificant cross-functional experience across engineering, sales, and marketing teams in cloud computing or related technical fields.\nProven successful program outcomes from idea to launch in multiple contexts throughout your career.\nAbility to manage the expectations, demands and priorities of multiple internal stakeholders based on overarching vision and success for global team health.\nAbility to work under pressure and possess flexibility with changing needs and direction in a rapidly-growing organization.\nStrong organization and communication skills."
Google,"Supplier Development Engineer (SDE), Cable/Connector",Manufacturing & Supply Chain,"Shanghai, China","Drive cross-functional activities in the supply chain for overall Technical Operational readiness in all NPI phases leading into mass production.\nCollaborate with suppliers and Engineering teams in assessing process technologies based on project requirements, and propose and develop the manufacturing blueprint including process flow, equipment/fixture designs, implementation schedules and validation plans for engineering builds and mass production.\nDrive project technical and operational issues with material, process, fixtures, equipment, etc. during the NPI phase to enable delivery of a mature product and manufacturing process into mass production. Support/drive continuous improvement efforts in the supply chain.\nLead suppliers by providing technical direction to establish and validate (utilizing statistical tools) process capability during the NPI phase for consistently delivering a quality product in mass production.\nUtilize DOE’s, FMEA and other Industry standard tools to proactively identify and address risks and optimize process parameters.","BS degree in an Engineering discipline or equivalent practical experience.\n7 years of experience in Cable/Connector Design or Manufacturing in an NPI role.\nExperience working with Interconnect Engineering and Product Design (PD)/Mechanical Engineer in developing, manufacturing and testing.\nAbility to speak and write in English and Mandarin fluently and idiomatically.","BSEE, BSME or BSIE degree.\nExperience of using Statistics tools for Data analysis, e.g. distribution histogram/pareto chart, process control chart, Design of Experiment (DOE), Correlation Analysis, etc.\nDemonstrated knowledge in PCBA manufacturing process and quality control.\nFamilar with cable/connector related components' manufacturing: moldling, stamping, die-casting, LIM, MIM process and materials.\nSelf starter with innovation, integrity and attention to detail.\nAbility to travel up to 50% of the time"


1.4 Data Cleaning

Why remove category, location and responsibilities details ? 

- Our goal is to extract job roles and corresponding skills from the dataset. Location and category therefore is not relevant in our use case because skills required for certain job role stays the same regardless of the location. 
- As for responsibilities feature, same skill can be applied in several different domains performing several different responsibilities. Therefore, grouping job posts based on the responsiblities doesn't look like a good idea but it is worth experimenting with. In case the performance of the model is not upto the mark, grouping job postings based on responsiblities feature is a good idea.   

Why use basic qualifications and preferred qualifications as the features  ? 

- Basic qualifications and prefered qualification highlight the skills required for the given job in succinct manner and make the task of extracting skills from job description easy. So, we are using these features.

In [85]:
print(f'Number of records in amazon dataset is {amazon.shape[0]} with {amazon.shape[1]} features')
print(f'Number of records in google dataset is {google.shape[0]} with {amazon.shape[1]} features\n\n')

# filtering only useful columns
amazon_filtered = amazon[['Title', 'BASIC QUALIFICATIONS', 'PREFERRED QUALIFICATIONS']]
google_filtered = google[['Title', 'Minimum Qualifications', 'Preferred Qualifications']]

amazon_filtered.reset_index(drop=True, inplace=True)
google_filtered.reset_index(drop=True, inplace=True)

google_filtered.rename(columns={'Title':'title', 'Minimum Qualifications': 'basic_qualif', 'Preferred Qualifications':'pref_qualif' }, inplace=True)
amazon_filtered.rename(columns={'Title':'title', 'BASIC QUALIFICATIONS' : 'basic_qualif', 'PREFERRED QUALIFICATIONS' : 'pref_qualif'}, inplace=True)

# combine the data from amazon and google job postings
data = pd.concat([google_filtered, amazon_filtered], axis=0)
data.reset_index(drop=True, inplace=True)

for col in data.columns : 
  data[col] = data[col].str.lower()
uncleaned_data = data.copy()
uncleaned_data.to_csv('unclean_data.csv')
data.head()

Number of records in amazon dataset is 3493 with 6 features
Number of records in google dataset is 1250 with 6 features




<ipython-input-85-807bc5b00ed7>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  google_filtered.rename(columns={'Title':'title', 'Minimum Qualifications': 'basic_qualif', 'Preferred Qualifications':'pref_qualif' }, inplace=True)
<ipython-input-85-807bc5b00ed7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amazon_filtered.rename(columns={'Title':'title', 'BASIC QUALIFICATIONS' : 'basic_qualif', 'PREFERRED QUALIFICATIONS' : 'pref_qualif'}, inplace=True)


,title,basic_qualif,pref_qualif
0,google cloud program manager,"ba/bs degree or equivalent practical experience.\n3 years of experience in program and/or project management in cloud computing, enterprise software and/or marketing technologies.","experience in the business technology market as a program manager in saas, cloud computing, and/or emerging technologies.\nsignificant cross-functional experience across engineering, sales, and marketing teams in cloud computing or related technical fields.\nproven successful program outcomes from idea to launch in multiple contexts throughout your career.\nability to manage the expectations, demands and priorities of multiple internal stakeholders based on overarching vision and success for global team health.\nability to work under pressure and possess flexibility with changing needs and direction in a rapidly-growing organization.\nstrong organization and communication skills."
1,"supplier development engineer (sde), cable/connector","bs degree in an engineering discipline or equivalent practical experience.\n7 years of experience in cable/connector design or manufacturing in an npi role.\nexperience working with interconnect engineering and product design (pd)/mechanical engineer in developing, manufacturing and testing.\nability to speak and write in english and mandarin fluently and idiomatically.","bsee, bsme or bsie degree.\nexperience of using statistics tools for data analysis, e.g. distribution histogram/pareto chart, process control chart, design of experiment (doe), correlation analysis, etc.\ndemonstrated knowledge in pcba manufacturing process and quality control.\nfamilar with cable/connector related components' manufacturing: moldling, stamping, die-casting, lim, mim process and materials.\nself starter with innovation, integrity and attention to detail.\nability to travel up to 50% of the time"
2,"data analyst, product and tools operations, google technical services","bachelor’s degree in business, economics, statistics, operations research or related analytical field, or equivalent practical experience.\n2 years of work experience in business analysis.\n1 year of experience with statistical modeling, forecasting or machine learning. experience with r, sas or python.\n1 year of experience developing and manipulating large datasets.",experience partnering or consulting cross-functionally with senior stakeholders.\nproficiency in a database query language (e.g. sql).\nability to manage multiple projects in an ambiguous environment.\nstrong presentation and communication skills with the ability to communicate statistical concepts and explain recommendations to non-experts.
3,"developer advocate, partner engineering","ba/bs degree in computer science or equivalent practical experience.\nexperience working directly with partners.\nprogramming experience in one or more of the following languages/platforms: android, java, kotlin, ios, javascript.","experience as a software developer, architect, technology advocate, cto, or consultant working with web or mobile technologies.\nexperience working with third parties.\nexperience interacting with clients or internal stakeholders.\nknowledge of web application or mobile application development landscapes."
4,"program manager, audio visual (av) deployments","ba/bs degree or equivalent practical experience.\n4 years of experience managing large scale global av deployments.\nexperience managing vendors to deliver defined projects.\nexperience in audio/visual and video conferencing design, system integration and resolution.",cts certification.\nexperience in the construction sector.\nexperience with project management software and reporting tools.\nability to understand technical subjects and emerging technologies and their relevance to the marketplace.\nbusiness management and consulting skills.\nexcellent interpersonal and relationship building skills across multiple stakeholders at varying levels.


In [86]:
def clean_text(text):
  """
    Job descriptions often contain data that does not represent the role/job position's functions but instead highlights the company's culture and 
    related aspects. As these details don't really add any value for our use case of skills extraction, we will remove them all from our data records
    input : uncleaned text 
    output : cleaned text without company name, punctuations, etc. Note that not all punctuation symbols are removed as they convey some meaning 
             and embeddings are capable of representing the same
  """
  filters = [
      r"amazon",
      r"http://aws.amazon.com",
      r"inc. group of companies",
      r"is an equal opportunity-affirmative action employer",
      r"minority / female / disability / veteran / gender identity / sexual orientation",
      r"come and join us",
      r"we believe passionately that employing a diverse workforce is central to our success",
      r"we make recruiting decisions based on your experience and skills",
      r"we value your passion to discover, invent, simplify and build",
      r"for more information on amazon web services please visit",
      r"#[A-Za-z0-9_]+",
      r'\n',
      r':-',
      r' - ',
      r'/', 
      r'\·',
      r'\*',
      r'\s+',
      r'e\.g\.',
      r'•',
      r'–'
  ]
  for pattern in filters: 
    try : 
      text = re.sub(pattern, ' ', text) 
    except : 
      print(pattern, text)
  text = text.strip()
  return text

def pre_process(text):
  """
    Does preprocessing on text like replacing the acronyms with their full forms.
    input : text 
    output : processed text with acronyms replaced by their full forms.
  """
  acronyms = {
      'sr.' : 'senior'
  }
  for key, value in acronyms.items() : 
    text = re.sub(key, value, text)
  return text

# Check for presence of null values
data.isna().sum()

title            0
basic_qualif    23
pref_qualif     23
dtype: int64

In [87]:
# As the number of Null values is very small as compared to our dataset, we can safely drop them. 
# drop nan values 
data.dropna(inplace=True)

# clean data
data = data.applymap(clean_text)
data = data.applymap(pre_process)

data.sample(3)

,title,basic_qualif,pref_qualif
620,"bold intern, summer 2018","be currently enrolled in a 4-year ba bs program, in any major, at a university in the united states with an expected graduation date between december 2018 and june 2019. be able to complete an 11-week program beginning in may or june 2018. have authorization to work in the united states.","analytical skills and an interest in taking on business problems. interpersonal and organizational skills, with the ability to navigate an ambiguous environment. interest in the technology industry and google’s business."
2944,"software development engineers worldwide transportation engineering @ hyderabad, india","bachelor’s degree or higher in computer science, engineering or related field 5+ years professional experience in commercial or web-scale system software development and delivery proven object-oriented design and implementation skills (java, and or c++), preferably on linux proven understanding of scalable computing systems, software architecture, data structures and algorithms proficient in network, distributed, asynchronous and concurrent programming experience designing, building and delivering high quality software on time solid verbal and written communication skills",experience in agile scrum enterprise-scale software development experience building scalable infrastructure software or distributed systems demonstrated ability to achieve goals in a highly innovative and fast paced environment
753,"mba intern, summer 2018",must be pursuing a full-time mba program with an expected graduation date between december 2018 and june 2019. have authorization to work in the united states. be able to complete a 11-12 week program beginning in may or june 2018.,"interest in the technology industry and google’s business. demonstrated critical thinking skills and independent motivation. excellent project management, interpersonal and organizational skills, with the ability to drive and implement ideas in a changing environment. analytical skills, and ability to solve highly strategic and difficult business problems."


In [91]:
data.to_csv('clean_data.csv', index=False)